In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Cnn_classification/cnn_img/')

##비디오 프레임별로 저장

In [ ]:
import cv2
vidcap = cv2.VideoCapture('/content/drive/MyDrive/video/KakaoTalk_20220331_201401471.mp4') # 비디오 파일 경로
success,image = vidcap.read()

count = 0
success = True

while success:
  success,image = vidcap.read()
  cv2.imwrite("/content/drive/MyDrive/video/img/%d.jpg" % count, image) # 저장할 폴더와 이름 지정
  print("saved image %d.jpg" % count)
  
  if cv2.waitKey(10) == 27:                    
      break
  count += 1

##Face recognition

In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=85984f2370295b918e58983e6f6e86d4e3b758019ef84b403a51f57a74a270ad
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
from PIL import Image
import face_recognition as frc
import time
import cv2

groups_folder_path = 'original/'
start = time.time()

image_w = 224
image_h = 224
categories = ["경한", "민용", "연수", "원", "유리", "혜령"]

# 얼굴 인식 후 인식된 얼굴 저장
for idex, categorie in enumerate(categories):
    image_dir = groups_folder_path + categorie + '/'
    
    # image_dir 경로의 모든 파일들에 대하여 작업
    for top, dir, f in os.walk(image_dir):
        # 경로내 모든 파일에 대하여
        for filename in f:
            print(image_dir+filename)
            
            # 파일불러오기
            img = frc.load_image_file(image_dir+filename)
            
            # 파일 얼굴인식
            faces = frc.face_locations(img)
            
            # 얼굴 인식
            if len(faces) == 1:
                for face_location in faces:
                    top, right, bottom, left = face_location
                    face_image = img[top:bottom, left:right]
                    resize_img = cv2.resize(face_image, dsize = (224,224))
                    rgb_img = cv2.cvtColor(resize_img, cv2.COLOR_BGR2RGB)
                    cv2.imwrite('face_only/'+ categorie + '/' + filename, rgb_img)

original/경한/KakaoTalk_20220405_235828458.png
original/경한/KakaoTalk_20220405_235828583.png
original/경한/KakaoTalk_20220405_235828691.png
original/경한/KakaoTalk_20220405_235829031.png
original/경한/KakaoTalk_20220405_235829224.png
original/경한/KakaoTalk_20220405_235829389.png
original/경한/KakaoTalk_20220405_235829639.png
original/경한/KakaoTalk_20220405_235829888.png
original/경한/KakaoTalk_20220406_000604303.png
original/경한/7.JPG
original/민용/KakaoTalk_20220405_235811769_02.jpg
original/민용/KakaoTalk_20220405_235811769_06.jpg
original/민용/KakaoTalk_20220405_235811769_05.jpg
original/민용/KakaoTalk_20220405_235811769_07.jpg
original/민용/KakaoTalk_20220405_235811769_04.jpg
original/민용/KakaoTalk_20220405_235824294.jpg
original/민용/1.JPG
original/민용/2.JPG
original/민용/3.JPG
original/민용/4.JPG
original/연수/KakaoTalk_20220405_235026140.jpg
original/연수/KakaoTalk_20220405_235027318.jpg
original/연수/KakaoTalk_20220405_235028478.jpg
original/연수/KakaoTalk_20220405_235029303.jpg
original/연수/KakaoTalk_20220405_235029944

##Augmentation

In [ ]:
import face_recognition as frc
from keras.preprocessing.image import ImageDataGenerator
import os
Img_create = ImageDataGenerator(rotation_range = 30,  # 30도 이내의 회전
                                horizontal_flip=True, # 좌우반전
                                brightness_range = [0.6,1.0],  # 밝기조절
                                fill_mode='nearest')
face_dir = 'face_only/원/'  # Augmentation 적용할 이미지가 있는 폴더
i = 0
# image_dir 경로의 모든 파일들에 대하여 작업
for top, dir, f in os.walk(face_dir):
    # 경로내 모든 파일에 대하여
    for filename in f:
        print(face_dir+filename)
            
        # 파일불러오기
        face_img = frc.load_image_file(face_dir+filename)
        face_img = face_img.reshape((1,) + face_img.shape)
        
        for batch in Img_create.flow(face_img, batch_size = 1, 
                                     save_to_dir = face_dir, save_format = 'jpg', save_prefix = 'new_' + filename):
            i += 1
            if i % 5 == 0:
                i = 0
                break

face_only/원/KakaoTalk_20220406_003054429_08.jpg
face_only/원/KakaoTalk_20220406_003054429_05.jpg
face_only/원/KakaoTalk_20220406_003054429_01.jpg
face_only/원/KakaoTalk_20220406_003054429.jpg
face_only/원/KakaoTalk_20220406_003054429_07.jpg
face_only/원/KakaoTalk_20220406_003054429_02.jpg
face_only/원/KakaoTalk_20220406_003054429_09.jpg
face_only/원/KakaoTalk_20220406_003054429_06.jpg
face_only/원/KakaoTalk_20220406_003054429_04.jpg


## (+) Image crop

In [ ]:
import cv2
from PIL import Image 
import glob
import pandas as pd

count = 1
path = 'face_only/경한' # crop 적용할 이미지가 있는 폴더
filepath = glob.glob(path + '/*')

for i in range(len(filepath)):
    image1 = Image.open(filepath[i])
    croppedImage=image1.crop((10,10,180,180))
    croppedImage.save('crop/경한/croppedImage_%d.jpg'%count)
    count += 1